In [ ]:
import numpy as np
import glob2
import datetime
from pathlib import Path
from tqdm.notebook import tqdm
import pickle
from matplotlib import pyplot as plt
from utils.detection.association_geodesic import squarize
plt.rcParams.update({'font.size': 18})

from utils.physics.signal.make_spectrogram import make_spectrogram
from utils.data_reading.sound_data.station import StationsCatalog
import skimage
from utils.physics.sound_model.spherical_sound_model import GridSphericalSoundModel as GridSoundModel, HomogeneousSphericalSoundModel as HomogeneousSoundModel
from utils.detection.association_geodesic import compute_candidates, update_valid_grid, update_results, load_detections, compute_grids

In [ ]:
# paths
CATALOG_PATH = "/media/plerolland/akoustik/MAHY"
DETECTIONS_DIR = f"../../../../../data/detection/i_TiSSNet_checked-reboot-2"
OUT = "/media/plerolland/LaBoite/MAHY_eval/i-reboot-2"

STATIONS = StationsCatalog(CATALOG_PATH).filter_out_undated().filter_out_unlocated()

DELTA = datetime.timedelta(seconds=100)

In [ ]:
d = []
for station in tqdm(STATIONS):
    path = f"{DETECTIONS_DIR}/{station.dataset}/{station.dataset}_{station.name}.pkl"

    with open(path, "rb") as f:
        while True:
            try:
                d.append([station] + pickle.load(f))
            except EOFError:
                break
d = np.array(d)

print(len(d), d.shape)
print(d[0])

In [ ]:
d2 = d[d[:,2] >= 0.5]
print(len(d2))
np.random.seed(0)
np.random.shuffle(d2)

d2 = d2[:1_000]

In [ ]:
import os
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm
import skimage.io
import numpy as np

def process_detection(args):
    manager, date, delta, path = args
    if Path(path).exists():
        return
    if date - delta < manager.dataset_start or date + delta > manager.dataset_end:
        print("skipping out of bounds segment")
        return
    data = manager.get_segment(date - delta, date + delta)
    spectrogram = make_spectrogram(
        data,
        manager.sampling_f,
        t_res=0.25,
        f_res=2,
        return_bins=False,
        normalize=False
    ).astype(np.float32)

    # Normalisation
    spectrogram = (spectrogram - np.mean(spectrogram)) / np.std(spectrogram)
    spectrogram = 255 * (spectrogram - np.min(spectrogram)) / (np.max(spectrogram) - np.min(spectrogram))

    # Sauvegarde
    skimage.io.imsave(path, spectrogram.astype(np.uint8))

with ProcessPoolExecutor(max_workers=10) as executor:
    args_iterable = ((s.get_manager(), date, DELTA, f'{OUT}/{s.name}_{date.strftime("%Y%m%d_%H%M%S_%f")}.png') for s, date, p in d2)
    list(tqdm(executor.map(process_detection, args_iterable), total=len(d2)))

In [ ]:
import os
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm
import skimage.io
import numpy as np

def process_detection(args):
    manager, date, delta, path = args
    if Path(path).exists():
        return
    if date - delta < manager.dataset_start or date + delta > manager.dataset_end:
        #print("skipping out of bounds segment")
        return
    data = manager.get_segment(date - delta, date + delta)
    spectrogram = make_spectrogram(
        data,
        manager.sampling_f,
        t_res=0.25,
        f_res=2,
        return_bins=False,
        normalize=False
    ).astype(np.float32)

    # Normalisation
    spectrogram = (spectrogram - np.mean(spectrogram)) / np.std(spectrogram)
    spectrogram = 255 * (spectrogram - np.min(spectrogram)) / (np.max(spectrogram) - np.min(spectrogram))

    # Sauvegarde
    skimage.io.imsave(path, spectrogram.astype(np.uint8))

for s, date, p in detections_uncertain:
    process_detection((s.get_manager(), date, DELTA, f'{OUT}/{s.name}_{date.strftime("%Y%m%d_%H%M%S_%f")}.png'))